In [4]:
import pickle
import pandas as pd
import datetime
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras import applications
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Dropout, Flatten, Dense
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Activation
from DataGen_BinaryLabels import DataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras import backend
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, f1_score

#returns a list  of  hyperparamter settings
def load_hyperparameter_settings(sampler_file):
    hyp_list = []
    
    with open(sampler_file, "rb") as obj:
        for i in range(10):
            hyp_list.append(pickle.load(obj))
            
    return hyp_list

#returns VGG16 base_model
def load_base_model_vgg16(img_width,img_height,num_channels,weight_init='imagenet',include_fc_layers=False):
    base_model = applications.VGG16(weights=weight_init, input_shape=(img_width, img_height, num_channels)
                                    ,include_top=include_fc_layers)
    return base_model

#hyperparam_setting - dictionary of hyperparameter setting
#base_model - the base model to fine tune
#returns the full model after adding the FC layers to the base_model

def add_fc_layers(hyperparam_setting, base_model):
    num_layers = hyperparam_setting['layers']
    nodes_fc_layer_1 = int(hyperparam_setting['nodes_1'])
    
    print('Dense('+str(nodes_fc_layer_1)+'), use_bias = False)(X)')
    print('BatchNormalization()(X)')
    print('Activation(relu)(X)')
    print('Dropout(0.3)(X)')
    
    X = base_model.output
    X = Flatten(input_shape=base_model.output_shape[1:])(X)
    
    X = Dense(nodes_fc_layer_1, use_bias = False)(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    X = Dropout(0.3)(X)  
    
    if num_layers > 1:
        print('more than one layer')
        
        for n_layer in range(2,num_layers+1):
            #str(setting['nodes_'+str(n_layer)
            
            num_nodes = int(hyperparam_setting['nodes_'+str(n_layer)])
            X = Dense(num_nodes, use_bias = False)(X)
            X = BatchNormalization()(X)
            X = Activation('relu')(X)
            X = Dropout(0.3)(X)  
            
            print('Dense('+str(num_nodes)+'), use_bias = False)(X)')
            print('BatchNormalization()(X)')
            print('Activation(relu)(X)')
            print('Dropout(0.3)(X)')
    else:
        print('only one layer')
        
    Output = Dense(1, use_bias = False)(X) #number of dense nodes is 1 because it is a binary classification problem
    Output = BatchNormalization()(Output)
    Output = Activation('sigmoid')(Output)
    
    model = Model(inputs=base_model.input, outputs=Output)
    return model

#set the first num_layers to nontrainable
#model - an instance of Keras Model
# => model is the final model (base_model added with fully connected layers)

def set_nontrainable_layers(num_layers, model):
    for layer in model.layers[:num_layers]:
        layer.trainable = False
        
    return model

#model - an instance of Keras Model
# => model is the final model (base_model added with fully connected layers)

def add_loss_function(hyperparam_setting, model, metric = 'accuracy'):
    learning_rate = hyperparam_setting['learning_rate']
    
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizers.Adam(lr = learning_rate),
                  metrics=[metric])
    
    return model

#returns the dict of cross validation settings

def load_cross_validation_settings(cv_file):
    cv_setting = None
    with open(cv_file, "rb") as obj:
        cv_setting = pickle.load(obj)
        
    return cv_setting
    
    
    
def fit_generator(hyperparameter_setting, training_generator, 
                  model, setting_count, cv_fold_idx,
                  save_best_model_only=False):
    #print("==========")

    num_epochs = hyperparameter_setting['epochs']
    history = model.fit(
                training_generator,
                epochs = num_epochs,
                verbose=1
                )
    
    #print(datetime.datetime.now())
    
   # del checkpointer
    return history


def predict_data(model_list, cv_file, 
                         generator_params_dict):
        
        cv_setting = load_cross_validation_settings(cv_file)
        final_test = cv_setting['Test']
        subject_dict_test = {}
        subject_list_test = []
        
        
        for x,y in zip(final_test[0], final_test[1]):
            subject_list_test.append(x)
            subject_dict_test[x] = y
        
        testing_generator = DataGenerator(subject_list_test, subject_dict_test, **generator_params_dict)
        


        prediction_array = []
        accuracy_list = []
        confusion_matrix_list = []
        sensitivity_list = []
        specificity_list = []
        f1_score_list = []
        
        dim = params_dict['dim']
        Ch = params_dict['n_channels']
        
        model = load_model('Eight_layer_CNN.hdf5')
        cur_pred = model.predict_generator(testing_generator, verbose = 1)
        cur_pred_thres = []
        pred_list = []
            
        for pred in cur_pred:
                v = pred[0]
                pred_list.append(v)
                
                if v >= 0.5:
                    cur_pred_thres.append(1)
                else:
                    cur_pred_thres.append(0)
        prediction_array.append(pred_list)
            
            
        y_pred = np.array(cur_pred_thres)
        y_true = np.array(list(subject_dict_test.values()))
        acc = accuracy_score(y_pred, y_true)
        conf = confusion_matrix(y_pred, y_true)
        sen = conf[0,0]/(conf[0,0] + conf[1,0])
        spec = conf[1,1]/(conf[1,1] + conf[0,1])
        f1 = f1_score(y_pred, y_true)
            
        print('sen',sen)
        print('spec', spec)
        print('f1',f1)
        print('acc',acc)
           
            
        return [prediction_array, accuracy_list, confusion_matrix_list, sensitivity_list, specificity_list, f1_score_list]
img_width = 256
img_height = 256
num_channels = 3

history_path = r'/media/iitindmaths/Seagate_Expansion_Drive/DTE_ADNI_Baseline_Revision/CN_vs_AD/history'
cv_file = '/media/iitindmaths/Seagate_Expansion_Drive/DTE_ADNI_Baseline_CN_vs_AD_CV_File.pickle'
model_list_path = r'/media/iitindmaths/Seagate_Expansion_Drive/DTE_ADNI_Baseline_Revision/CN_vs_AD/final_models'
model_list = []
full_model_list = []


    
params_dict = {'dim':(img_width,img_height),
                   'batch_size': 1,
                   'n_classes': 2,
                   'n_channels': num_channels,
                   'shuffle':False
                  }
result_list = predict_data([], cv_file, params_dict)


2656/2656 [==============================] - 9s 3ms/step
sen 0.8917066483893078
spec 0.849624060150376
f1 0.8575042158516021
acc 0.8727409638554217


In [2]:
import pickle
import pandas as pd
import datetime
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras import applications
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Dropout, Flatten, Dense
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Activation
from DataGen_BinaryLabels import DataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras import backend
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, f1_score

#returns a list  of  hyperparamter settings
def load_hyperparameter_settings(sampler_file):
    hyp_list = []
    
    with open(sampler_file, "rb") as obj:
        for i in range(10):
            hyp_list.append(pickle.load(obj))
            
    return hyp_list

#returns VGG16 base_model
def load_base_model_vgg16(img_width,img_height,num_channels,weight_init='imagenet',include_fc_layers=False):
    base_model = applications.VGG16(weights=weight_init, input_shape=(img_width, img_height, num_channels)
                                    ,include_top=include_fc_layers)
    return base_model

#hyperparam_setting - dictionary of hyperparameter setting
#base_model - the base model to fine tune
#returns the full model after adding the FC layers to the base_model

def add_fc_layers(hyperparam_setting, base_model):
    num_layers = hyperparam_setting['layers']
    nodes_fc_layer_1 = int(hyperparam_setting['nodes_1'])
    
    print('Dense('+str(nodes_fc_layer_1)+'), use_bias = False)(X)')
    print('BatchNormalization()(X)')
    print('Activation(relu)(X)')
    print('Dropout(0.3)(X)')
    
    X = base_model.output
    X = Flatten(input_shape=base_model.output_shape[1:])(X)
    
    X = Dense(nodes_fc_layer_1, use_bias = False)(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    X = Dropout(0.3)(X)  
    
    if num_layers > 1:
        print('more than one layer')
        
        for n_layer in range(2,num_layers+1):
            #str(setting['nodes_'+str(n_layer)
            
            num_nodes = int(hyperparam_setting['nodes_'+str(n_layer)])
            X = Dense(num_nodes, use_bias = False)(X)
            X = BatchNormalization()(X)
            X = Activation('relu')(X)
            X = Dropout(0.3)(X)  
            
            print('Dense('+str(num_nodes)+'), use_bias = False)(X)')
            print('BatchNormalization()(X)')
            print('Activation(relu)(X)')
            print('Dropout(0.3)(X)')
    else:
        print('only one layer')
        
    Output = Dense(1, use_bias = False)(X) #number of dense nodes is 1 because it is a binary classification problem
    Output = BatchNormalization()(Output)
    Output = Activation('sigmoid')(Output)
    
    model = Model(inputs=base_model.input, outputs=Output)
    return model

#set the first num_layers to nontrainable
#model - an instance of Keras Model
# => model is the final model (base_model added with fully connected layers)

def set_nontrainable_layers(num_layers, model):
    for layer in model.layers[:num_layers]:
        layer.trainable = False
        
    return model

#model - an instance of Keras Model
# => model is the final model (base_model added with fully connected layers)

def add_loss_function(hyperparam_setting, model, metric = 'accuracy'):
    learning_rate = hyperparam_setting['learning_rate']
    
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizers.Adam(lr = learning_rate),
                  metrics=[metric])
    
    return model

#returns the dict of cross validation settings

def load_cross_validation_settings(cv_file):
    cv_setting = None
    with open(cv_file, "rb") as obj:
        cv_setting = pickle.load(obj)
        
    return cv_setting
    
    
    
def fit_generator(hyperparameter_setting, training_generator, 
                  model, setting_count, cv_fold_idx,
                  save_best_model_only=False):
    #print("==========")

    num_epochs = hyperparameter_setting['epochs']
    history = model.fit(
                training_generator,
                epochs = num_epochs,
                verbose=1
                )
    
    #print(datetime.datetime.now())
    
   # del checkpointer
    return history


def predict_data(model_list, cv_file, 
                         generator_params_dict):
        
        cv_setting = load_cross_validation_settings(cv_file)
        final_test = cv_setting['Test']
        subject_dict_test = {}
        subject_list_test = []
        
        
        for x,y in zip(final_test[0], final_test[1]):
            subject_list_test.append(x)
            subject_dict_test[x] = y
        
        testing_generator = DataGenerator(subject_list_test, subject_dict_test, **generator_params_dict)
        


        prediction_array = []
        accuracy_list = []
        confusion_matrix_list = []
        sensitivity_list = []
        specificity_list = []
        f1_score_list = []
        
        dim = params_dict['dim']
        Ch = params_dict['n_channels']
        
        model = load_model('Eight_layer_CNN_MCI_vs_AD.hdf5')
        cur_pred = model.predict_generator(testing_generator, verbose = 1)
        cur_pred_thres = []
        pred_list = []
            
        for pred in cur_pred:
                v = pred[0]
                pred_list.append(v)
                
                if v >= 0.5:
                    cur_pred_thres.append(1)
                else:
                    cur_pred_thres.append(0)
        prediction_array.append(pred_list)
            
            
        y_pred = np.array(cur_pred_thres)
        y_true = np.array(list(subject_dict_test.values()))
        acc = accuracy_score(y_pred, y_true)
        conf = confusion_matrix(y_pred, y_true)
        sen = conf[0,0]/(conf[0,0] + conf[1,0])
        spec = conf[1,1]/(conf[1,1] + conf[0,1])
        f1 = f1_score(y_pred, y_true)
            
        print('sen',sen)
        print('spec', spec)
        print('f1',f1)
        print('acc',acc)
           
            
        return [prediction_array, accuracy_list, confusion_matrix_list, sensitivity_list, specificity_list, f1_score_list]
img_width = 256
img_height = 256
num_channels = 3

history_path = r'/media/iitindmaths/Seagate_Expansion_Drive/DTE_ADNI_Baseline_Revision/MCI_vs_AD/history'
cv_file = '/media/iitindmaths/Seagate_Expansion_Drive/DTE_ADNI_Baseline_MCI_vs_AD_CV_File.pickle'
model_list_path = r'/media/iitindmaths/Seagate_Expansion_Drive/DTE_ADNI_Baseline_Revision/MCI_vs_AD/final_models'
model_list = []
full_model_list = []


    
params_dict = {'dim':(img_width,img_height),
                   'batch_size': 1,
                   'n_classes': 2,
                   'n_channels': num_channels,
                   'shuffle':False
                  }
result_list = predict_data([], cv_file, params_dict)


3744/3744 [==============================] - 29s 8ms/step
sen 0.9623085983510011
spec 0.620718462823726
f1 0.7298624754420432
acc 0.8530982905982906


In [1]:
import pickle
import pandas as pd
import datetime
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras import applications
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Dropout, Flatten, Dense
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Activation
from DataGen_BinaryLabels import DataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras import backend
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, f1_score

#returns a list  of  hyperparamter settings
def load_hyperparameter_settings(sampler_file):
    hyp_list = []
    
    with open(sampler_file, "rb") as obj:
        for i in range(10):
            hyp_list.append(pickle.load(obj))
            
    return hyp_list

#returns VGG16 base_model
def load_base_model_vgg16(img_width,img_height,num_channels,weight_init='imagenet',include_fc_layers=False):
    base_model = applications.VGG16(weights=weight_init, input_shape=(img_width, img_height, num_channels)
                                    ,include_top=include_fc_layers)
    return base_model

#hyperparam_setting - dictionary of hyperparameter setting
#base_model - the base model to fine tune
#returns the full model after adding the FC layers to the base_model

def add_fc_layers(hyperparam_setting, base_model):
    num_layers = hyperparam_setting['layers']
    nodes_fc_layer_1 = int(hyperparam_setting['nodes_1'])
    
    print('Dense('+str(nodes_fc_layer_1)+'), use_bias = False)(X)')
    print('BatchNormalization()(X)')
    print('Activation(relu)(X)')
    print('Dropout(0.3)(X)')
    
    X = base_model.output
    X = Flatten(input_shape=base_model.output_shape[1:])(X)
    
    X = Dense(nodes_fc_layer_1, use_bias = False)(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    X = Dropout(0.3)(X)  
    
    if num_layers > 1:
        print('more than one layer')
        
        for n_layer in range(2,num_layers+1):
            #str(setting['nodes_'+str(n_layer)
            
            num_nodes = int(hyperparam_setting['nodes_'+str(n_layer)])
            X = Dense(num_nodes, use_bias = False)(X)
            X = BatchNormalization()(X)
            X = Activation('relu')(X)
            X = Dropout(0.3)(X)  
            
            print('Dense('+str(num_nodes)+'), use_bias = False)(X)')
            print('BatchNormalization()(X)')
            print('Activation(relu)(X)')
            print('Dropout(0.3)(X)')
    else:
        print('only one layer')
        
    Output = Dense(1, use_bias = False)(X) #number of dense nodes is 1 because it is a binary classification problem
    Output = BatchNormalization()(Output)
    Output = Activation('sigmoid')(Output)
    
    model = Model(inputs=base_model.input, outputs=Output)
    return model

#set the first num_layers to nontrainable
#model - an instance of Keras Model
# => model is the final model (base_model added with fully connected layers)

def set_nontrainable_layers(num_layers, model):
    for layer in model.layers[:num_layers]:
        layer.trainable = False
        
    return model

#model - an instance of Keras Model
# => model is the final model (base_model added with fully connected layers)

def add_loss_function(hyperparam_setting, model, metric = 'accuracy'):
    learning_rate = hyperparam_setting['learning_rate']
    
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizers.Adam(lr = learning_rate),
                  metrics=[metric])
    
    return model

#returns the dict of cross validation settings

def load_cross_validation_settings(cv_file):
    cv_setting = None
    with open(cv_file, "rb") as obj:
        cv_setting = pickle.load(obj)
        
    return cv_setting
    
    
    
def fit_generator(hyperparameter_setting, training_generator, 
                  model, setting_count, cv_fold_idx,
                  save_best_model_only=False):
    #print("==========")

    num_epochs = hyperparameter_setting['epochs']
    history = model.fit(
                training_generator,
                epochs = num_epochs,
                verbose=1
                )
    
    #print(datetime.datetime.now())
    
   # del checkpointer
    return history


def predict_data(model_list, cv_file, 
                         generator_params_dict):
        
        cv_setting = load_cross_validation_settings(cv_file)
        final_test = cv_setting['Test']
        subject_dict_test = {}
        subject_list_test = []
        
        
        for x,y in zip(final_test[0], final_test[1]):
            subject_list_test.append(x)
            subject_dict_test[x] = y
        
        testing_generator = DataGenerator(subject_list_test, subject_dict_test, **generator_params_dict)
        


        prediction_array = []
        accuracy_list = []
        confusion_matrix_list = []
        sensitivity_list = []
        specificity_list = []
        f1_score_list = []
        
        dim = params_dict['dim']
        Ch = params_dict['n_channels']
        
        model = load_model('Eight_layer_CNN_MCI_vs_AD_2.hdf5')
        cur_pred = model.predict_generator(testing_generator, verbose = 1)
        cur_pred_thres = []
        pred_list = []
            
        for pred in cur_pred:
                v = pred[0]
                pred_list.append(v)
                
                if v >= 0.5:
                    cur_pred_thres.append(1)
                else:
                    cur_pred_thres.append(0)
        prediction_array.append(pred_list)
            
            
        y_pred = np.array(cur_pred_thres)
        y_true = np.array(list(subject_dict_test.values()))
        acc = accuracy_score(y_pred, y_true)
        conf = confusion_matrix(y_pred, y_true)
        sen = conf[0,0]/(conf[0,0] + conf[1,0])
        spec = conf[1,1]/(conf[1,1] + conf[0,1])
        f1 = f1_score(y_pred, y_true)
            
        print('sen',sen)
        print('spec', spec)
        print('f1',f1)
        print('acc',acc)
           
            
        return [prediction_array, accuracy_list, confusion_matrix_list, sensitivity_list, specificity_list, f1_score_list]
img_width = 256
img_height = 256
num_channels = 3

history_path = r'/media/iitindmaths/Seagate_Expansion_Drive/DTE_ADNI_Baseline_Revision/MCI_vs_AD/history'
cv_file = '/media/iitindmaths/Seagate_Expansion_Drive/DTE_ADNI_Baseline_MCI_vs_AD_CV_File.pickle'
model_list_path = r'/media/iitindmaths/Seagate_Expansion_Drive/DTE_ADNI_Baseline_Revision/MCI_vs_AD/final_models'
model_list = []
full_model_list = []


    
params_dict = {'dim':(img_width,img_height),
                   'batch_size': 1,
                   'n_classes': 2,
                   'n_channels': num_channels,
                   'shuffle':False
                  }
result_list = predict_data([], cv_file, params_dict)


Instructions for updating:
Please use Model.predict, which supports generators.
3744/3744 [==============================] - 13s 3ms/step
sen 0.6482135846093443
spec 0.6265664160401002
f1 0.5276116778051353
acc 0.6412927350427351


In [1]:
import pickle
import pandas as pd
import datetime
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras import applications
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Dropout, Flatten, Dense
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Activation
from DataGen_BinaryLabels import DataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras import backend
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, f1_score

#returns a list  of  hyperparamter settings
def load_hyperparameter_settings(sampler_file):
    hyp_list = []
    
    with open(sampler_file, "rb") as obj:
        for i in range(10):
            hyp_list.append(pickle.load(obj))
            
    return hyp_list

#returns VGG16 base_model
def load_base_model_vgg16(img_width,img_height,num_channels,weight_init='imagenet',include_fc_layers=False):
    base_model = applications.VGG16(weights=weight_init, input_shape=(img_width, img_height, num_channels)
                                    ,include_top=include_fc_layers)
    return base_model

#hyperparam_setting - dictionary of hyperparameter setting
#base_model - the base model to fine tune
#returns the full model after adding the FC layers to the base_model

def add_fc_layers(hyperparam_setting, base_model):
    num_layers = hyperparam_setting['layers']
    nodes_fc_layer_1 = int(hyperparam_setting['nodes_1'])
    
    print('Dense('+str(nodes_fc_layer_1)+'), use_bias = False)(X)')
    print('BatchNormalization()(X)')
    print('Activation(relu)(X)')
    print('Dropout(0.3)(X)')
    
    X = base_model.output
    X = Flatten(input_shape=base_model.output_shape[1:])(X)
    
    X = Dense(nodes_fc_layer_1, use_bias = False)(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    X = Dropout(0.3)(X)  
    
    if num_layers > 1:
        print('more than one layer')
        
        for n_layer in range(2,num_layers+1):
            #str(setting['nodes_'+str(n_layer)
            
            num_nodes = int(hyperparam_setting['nodes_'+str(n_layer)])
            X = Dense(num_nodes, use_bias = False)(X)
            X = BatchNormalization()(X)
            X = Activation('relu')(X)
            X = Dropout(0.3)(X)  
            
            print('Dense('+str(num_nodes)+'), use_bias = False)(X)')
            print('BatchNormalization()(X)')
            print('Activation(relu)(X)')
            print('Dropout(0.3)(X)')
    else:
        print('only one layer')
        
    Output = Dense(1, use_bias = False)(X) #number of dense nodes is 1 because it is a binary classification problem
    Output = BatchNormalization()(Output)
    Output = Activation('sigmoid')(Output)
    
    model = Model(inputs=base_model.input, outputs=Output)
    return model

#set the first num_layers to nontrainable
#model - an instance of Keras Model
# => model is the final model (base_model added with fully connected layers)

def set_nontrainable_layers(num_layers, model):
    for layer in model.layers[:num_layers]:
        layer.trainable = False
        
    return model

#model - an instance of Keras Model
# => model is the final model (base_model added with fully connected layers)

def add_loss_function(hyperparam_setting, model, metric = 'accuracy'):
    learning_rate = hyperparam_setting['learning_rate']
    
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizers.Adam(lr = learning_rate),
                  metrics=[metric])
    
    return model

#returns the dict of cross validation settings

def load_cross_validation_settings(cv_file):
    cv_setting = None
    with open(cv_file, "rb") as obj:
        cv_setting = pickle.load(obj)
        
    return cv_setting
    
    
    
def fit_generator(hyperparameter_setting, training_generator, 
                  model, setting_count, cv_fold_idx,
                  save_best_model_only=False):
    #print("==========")

    num_epochs = hyperparameter_setting['epochs']
    history = model.fit(
                training_generator,
                epochs = num_epochs,
                verbose=1
                )
    
    #print(datetime.datetime.now())
    
   # del checkpointer
    return history


def predict_data(model_list, cv_file, 
                         generator_params_dict):
        
        cv_setting = load_cross_validation_settings(cv_file)
        final_test = cv_setting['Test']
        subject_dict_test = {}
        subject_list_test = []
        
        
        for x,y in zip(final_test[0], final_test[1]):
            subject_list_test.append(x)
            subject_dict_test[x] = y
        
        testing_generator = DataGenerator(subject_list_test, subject_dict_test, **generator_params_dict)
        


        prediction_array = []
        accuracy_list = []
        confusion_matrix_list = []
        sensitivity_list = []
        specificity_list = []
        f1_score_list = []
        
        dim = params_dict['dim']
        Ch = params_dict['n_channels']
        
        model = load_model('Eight_layer_CNN_MCI_vs_AD_3.hdf5')
        cur_pred = model.predict_generator(testing_generator, verbose = 1)
        cur_pred_thres = []
        pred_list = []
            
        for pred in cur_pred:
                v = pred[0]
                pred_list.append(v)
                
                if v >= 0.5:
                    cur_pred_thres.append(1)
                else:
                    cur_pred_thres.append(0)
        prediction_array.append(pred_list)
            
            
        y_pred = np.array(cur_pred_thres)
        y_true = np.array(list(subject_dict_test.values()))
        acc = accuracy_score(y_pred, y_true)
        conf = confusion_matrix(y_pred, y_true)
        sen = conf[0,0]/(conf[0,0] + conf[1,0])
        spec = conf[1,1]/(conf[1,1] + conf[0,1])
        f1 = f1_score(y_pred, y_true)
            
        print('sen',sen)
        print('spec', spec)
        print('f1',f1)
        print('acc',acc)
           
            
        return [prediction_array, accuracy_list, confusion_matrix_list, sensitivity_list, specificity_list, f1_score_list]
img_width = 256
img_height = 256
num_channels = 3

history_path = r'/media/iitindmaths/Seagate_Expansion_Drive/DTE_ADNI_Baseline_Revision/MCI_vs_AD/history'
cv_file = '/media/iitindmaths/Seagate_Expansion_Drive/DTE_ADNI_Baseline_MCI_vs_AD_CV_File.pickle'
model_list_path = r'/media/iitindmaths/Seagate_Expansion_Drive/DTE_ADNI_Baseline_Revision/MCI_vs_AD/final_models'
model_list = []
full_model_list = []


    
params_dict = {'dim':(img_width,img_height),
                   'batch_size': 1,
                   'n_classes': 2,
                   'n_channels': num_channels,
                   'shuffle':False
                  }
result_list = predict_data([], cv_file, params_dict)


Instructions for updating:
Please use Model.predict, which supports generators.
3744/3744 [==============================] - 14s 4ms/step
sen 0.9151943462897526
spec 0.7126148705096074
f1 0.7528684907325683
acc 0.8504273504273504


In [1]:
import pickle
import pandas as pd
import datetime
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras import applications
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Dropout, Flatten, Dense
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Activation
from DataGen_BinaryLabels import DataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras import backend
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, f1_score

#returns a list  of  hyperparamter settings
def load_hyperparameter_settings(sampler_file):
    hyp_list = []
    
    with open(sampler_file, "rb") as obj:
        for i in range(10):
            hyp_list.append(pickle.load(obj))
            
    return hyp_list

#returns VGG16 base_model
def load_base_model_vgg16(img_width,img_height,num_channels,weight_init='imagenet',include_fc_layers=False):
    base_model = applications.VGG16(weights=weight_init, input_shape=(img_width, img_height, num_channels)
                                    ,include_top=include_fc_layers)
    return base_model

#hyperparam_setting - dictionary of hyperparameter setting
#base_model - the base model to fine tune
#returns the full model after adding the FC layers to the base_model

def add_fc_layers(hyperparam_setting, base_model):
    num_layers = hyperparam_setting['layers']
    nodes_fc_layer_1 = int(hyperparam_setting['nodes_1'])
    
    print('Dense('+str(nodes_fc_layer_1)+'), use_bias = False)(X)')
    print('BatchNormalization()(X)')
    print('Activation(relu)(X)')
    print('Dropout(0.3)(X)')
    
    X = base_model.output
    X = Flatten(input_shape=base_model.output_shape[1:])(X)
    
    X = Dense(nodes_fc_layer_1, use_bias = False)(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    X = Dropout(0.3)(X)  
    
    if num_layers > 1:
        print('more than one layer')
        
        for n_layer in range(2,num_layers+1):
            #str(setting['nodes_'+str(n_layer)
            
            num_nodes = int(hyperparam_setting['nodes_'+str(n_layer)])
            X = Dense(num_nodes, use_bias = False)(X)
            X = BatchNormalization()(X)
            X = Activation('relu')(X)
            X = Dropout(0.3)(X)  
            
            print('Dense('+str(num_nodes)+'), use_bias = False)(X)')
            print('BatchNormalization()(X)')
            print('Activation(relu)(X)')
            print('Dropout(0.3)(X)')
    else:
        print('only one layer')
        
    Output = Dense(1, use_bias = False)(X) #number of dense nodes is 1 because it is a binary classification problem
    Output = BatchNormalization()(Output)
    Output = Activation('sigmoid')(Output)
    
    model = Model(inputs=base_model.input, outputs=Output)
    return model

#set the first num_layers to nontrainable
#model - an instance of Keras Model
# => model is the final model (base_model added with fully connected layers)

def set_nontrainable_layers(num_layers, model):
    for layer in model.layers[:num_layers]:
        layer.trainable = False
        
    return model

#model - an instance of Keras Model
# => model is the final model (base_model added with fully connected layers)

def add_loss_function(hyperparam_setting, model, metric = 'accuracy'):
    learning_rate = hyperparam_setting['learning_rate']
    
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizers.Adam(lr = learning_rate),
                  metrics=[metric])
    
    return model

#returns the dict of cross validation settings

def load_cross_validation_settings(cv_file):
    cv_setting = None
    with open(cv_file, "rb") as obj:
        cv_setting = pickle.load(obj)
        
    return cv_setting
    
    
    
def fit_generator(hyperparameter_setting, training_generator, 
                  model, setting_count, cv_fold_idx,
                  save_best_model_only=False):
    #print("==========")

    num_epochs = hyperparameter_setting['epochs']
    history = model.fit(
                training_generator,
                epochs = num_epochs,
                verbose=1
                )
    
    #print(datetime.datetime.now())
    
   # del checkpointer
    return history


def predict_data(model_list, cv_file, 
                         generator_params_dict):
        
        cv_setting = load_cross_validation_settings(cv_file)
        final_test = cv_setting['Test']
        subject_dict_test = {}
        subject_list_test = []
        
        
        for x,y in zip(final_test[0], final_test[1]):
            subject_list_test.append(x)
            subject_dict_test[x] = y
        
        testing_generator = DataGenerator(subject_list_test, subject_dict_test, **generator_params_dict)
        


        prediction_array = []
        accuracy_list = []
        confusion_matrix_list = []
        sensitivity_list = []
        specificity_list = []
        f1_score_list = []
        
        dim = params_dict['dim']
        Ch = params_dict['n_channels']
        
        model = load_model('Eight_layer_CNN_MCI_vs_AD_4.hdf5')
        cur_pred = model.predict_generator(testing_generator, verbose = 1)
        cur_pred_thres = []
        pred_list = []
            
        for pred in cur_pred:
                v = pred[0]
                pred_list.append(v)
                
                if v >= 0.5:
                    cur_pred_thres.append(1)
                else:
                    cur_pred_thres.append(0)
        prediction_array.append(pred_list)
            
            
        y_pred = np.array(cur_pred_thres)
        y_true = np.array(list(subject_dict_test.values()))
        acc = accuracy_score(y_pred, y_true)
        conf = confusion_matrix(y_pred, y_true)
        sen = conf[0,0]/(conf[0,0] + conf[1,0])
        spec = conf[1,1]/(conf[1,1] + conf[0,1])
        f1 = f1_score(y_pred, y_true)
            
        print('sen',sen)
        print('spec', spec)
        print('f1',f1)
        print('acc',acc)
           
            
        return [prediction_array, accuracy_list, confusion_matrix_list, sensitivity_list, specificity_list, f1_score_list]
img_width = 256
img_height = 256
num_channels = 3

history_path = r'/media/iitindmaths/Seagate_Expansion_Drive/DTE_ADNI_Baseline_Revision/MCI_vs_AD/history'
cv_file = '/media/iitindmaths/Seagate_Expansion_Drive/DTE_ADNI_Baseline_MCI_vs_AD_CV_File.pickle'
model_list_path = r'/media/iitindmaths/Seagate_Expansion_Drive/DTE_ADNI_Baseline_Revision/MCI_vs_AD/final_models'
model_list = []
full_model_list = []


    
params_dict = {'dim':(img_width,img_height),
                   'batch_size': 1,
                   'n_classes': 2,
                   'n_channels': num_channels,
                   'shuffle':False
                  }
result_list = predict_data([], cv_file, params_dict)


Instructions for updating:
Please use Model.predict, which supports generators.
3744/3744 [==============================] - 13s 3ms/step
sen 0.9348252846486063
spec 0.4152046783625731
f1 0.5344086021505376
acc 0.7686965811965812


In [1]:
import pickle
import pandas as pd
import datetime
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras import applications
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Dropout, Flatten, Dense
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Activation
from DataGen_BinaryLabels import DataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras import backend
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, f1_score

#returns a list  of  hyperparamter settings
def load_hyperparameter_settings(sampler_file):
    hyp_list = []
    
    with open(sampler_file, "rb") as obj:
        for i in range(10):
            hyp_list.append(pickle.load(obj))
            
    return hyp_list

#returns VGG16 base_model
def load_base_model_vgg16(img_width,img_height,num_channels,weight_init='imagenet',include_fc_layers=False):
    base_model = applications.VGG16(weights=weight_init, input_shape=(img_width, img_height, num_channels)
                                    ,include_top=include_fc_layers)
    return base_model

#hyperparam_setting - dictionary of hyperparameter setting
#base_model - the base model to fine tune
#returns the full model after adding the FC layers to the base_model

def add_fc_layers(hyperparam_setting, base_model):
    num_layers = hyperparam_setting['layers']
    nodes_fc_layer_1 = int(hyperparam_setting['nodes_1'])
    
    print('Dense('+str(nodes_fc_layer_1)+'), use_bias = False)(X)')
    print('BatchNormalization()(X)')
    print('Activation(relu)(X)')
    print('Dropout(0.3)(X)')
    
    X = base_model.output
    X = Flatten(input_shape=base_model.output_shape[1:])(X)
    
    X = Dense(nodes_fc_layer_1, use_bias = False)(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    X = Dropout(0.3)(X)  
    
    if num_layers > 1:
        print('more than one layer')
        
        for n_layer in range(2,num_layers+1):
            #str(setting['nodes_'+str(n_layer)
            
            num_nodes = int(hyperparam_setting['nodes_'+str(n_layer)])
            X = Dense(num_nodes, use_bias = False)(X)
            X = BatchNormalization()(X)
            X = Activation('relu')(X)
            X = Dropout(0.3)(X)  
            
            print('Dense('+str(num_nodes)+'), use_bias = False)(X)')
            print('BatchNormalization()(X)')
            print('Activation(relu)(X)')
            print('Dropout(0.3)(X)')
    else:
        print('only one layer')
        
    Output = Dense(1, use_bias = False)(X) #number of dense nodes is 1 because it is a binary classification problem
    Output = BatchNormalization()(Output)
    Output = Activation('sigmoid')(Output)
    
    model = Model(inputs=base_model.input, outputs=Output)
    return model

#set the first num_layers to nontrainable
#model - an instance of Keras Model
# => model is the final model (base_model added with fully connected layers)

def set_nontrainable_layers(num_layers, model):
    for layer in model.layers[:num_layers]:
        layer.trainable = False
        
    return model

#model - an instance of Keras Model
# => model is the final model (base_model added with fully connected layers)

def add_loss_function(hyperparam_setting, model, metric = 'accuracy'):
    learning_rate = hyperparam_setting['learning_rate']
    
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizers.Adam(lr = learning_rate),
                  metrics=[metric])
    
    return model

#returns the dict of cross validation settings

def load_cross_validation_settings(cv_file):
    cv_setting = None
    with open(cv_file, "rb") as obj:
        cv_setting = pickle.load(obj)
        
    return cv_setting
    
    
    
def fit_generator(hyperparameter_setting, training_generator, 
                  model, setting_count, cv_fold_idx,
                  save_best_model_only=False):
    #print("==========")

    num_epochs = hyperparameter_setting['epochs']
    history = model.fit(
                training_generator,
                epochs = num_epochs,
                verbose=1
                )
    
    #print(datetime.datetime.now())
    
   # del checkpointer
    return history


def predict_data(model_list, cv_file, 
                         generator_params_dict):
        
        cv_setting = load_cross_validation_settings(cv_file)
        final_test = cv_setting['Test']
        subject_dict_test = {}
        subject_list_test = []
        
        
        for x,y in zip(final_test[0], final_test[1]):
            subject_list_test.append(x)
            subject_dict_test[x] = y
        
        testing_generator = DataGenerator(subject_list_test, subject_dict_test, **generator_params_dict)
        


        prediction_array = []
        accuracy_list = []
        confusion_matrix_list = []
        sensitivity_list = []
        specificity_list = []
        f1_score_list = []
        
        dim = params_dict['dim']
        Ch = params_dict['n_channels']
        
        model = load_model('Eight_layer_CNN_CN_vs_AD_4.hdf5')
        cur_pred = model.predict_generator(testing_generator, verbose = 1)
        cur_pred_thres = []
        pred_list = []
            
        for pred in cur_pred:
                v = pred[0]
                pred_list.append(v)
                
                if v >= 0.5:
                    cur_pred_thres.append(1)
                else:
                    cur_pred_thres.append(0)
        prediction_array.append(pred_list)
            
            
        y_pred = np.array(cur_pred_thres)
        y_true = np.array(list(subject_dict_test.values()))
        acc = accuracy_score(y_pred, y_true)
        conf = confusion_matrix(y_pred, y_true)
        sen = conf[0,0]/(conf[0,0] + conf[1,0])
        spec = conf[1,1]/(conf[1,1] + conf[0,1])
        f1 = f1_score(y_pred, y_true)
            
        print('sen',sen)
        print('spec', spec)
        print('f1',f1)
        print('acc',acc)
           
            
        return [prediction_array, accuracy_list, confusion_matrix_list, sensitivity_list, specificity_list, f1_score_list]
img_width = 256
img_height = 256
num_channels = 3

history_path = r'/media/iitindmaths/Seagate_Expansion_Drive/DTE_ADNI_Baseline_Revision/CN_vs_AD/history'
cv_file = '/media/iitindmaths/Seagate_Expansion_Drive/DTE_ADNI_Baseline_CN_vs_AD_CV_File.pickle'
model_list_path = r'/media/iitindmaths/Seagate_Expansion_Drive/DTE_ADNI_Baseline_Revision/CN_vs_AD/final_models'
model_list = []
full_model_list = []


    
params_dict = {'dim':(img_width,img_height),
                   'batch_size': 1,
                   'n_classes': 2,
                   'n_channels': num_channels,
                   'shuffle':False
                  }
result_list = predict_data([], cv_file, params_dict)


Instructions for updating:
Please use Model.predict, which supports generators.
2656/2656 [==============================] - 10s 4ms/step
sen 0.8766278272789582
spec 0.7050960735171261
f1 0.760018009905448
acc 0.7993222891566265
